In [0]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [3]:
link = 'https://drive.google.com/open?id=1lLa0K2SmZubQ65uoWJdSwv5mQV1QSetd'
file_name='data'
fluff, id = link.split('=')
# print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile(file_name)  
# from sklearn.externals import joblib
dataset=pd.read_csv(file_name, header=None)
dataset.head(5)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,43,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
# max(dataset.iloc[2, 1:])
# Y[0]

In [5]:
X = dataset.iloc[:, 1:].values
X=X.reshape((112800, 28*28))
y = dataset.iloc[:, 0].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#labelencoder_X_1 = LabelEncoder()
#X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
#labelencoder_X_2 = LabelEncoder()
#X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [0])
Y = onehotencoder.fit_transform(y.reshape(-1, 1)).toarray()
#X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
X_train=X
y_train=Y.astype("int")

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
#XX_test = sc.transform(XX_test)

X_train = X_train.reshape(-1, 28, 28, 1).astype("float32")
#XX_test = XX_test.reshape(-1, 28, 28, 1).astype("float32")


# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout
from keras.callbacks import LearningRateScheduler

classifier = Sequential()
 
classifier.add(Convolution2D(filters = 64, kernel_size = (5, 5), activation='relu', input_shape = (28, 28, 1)))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters = 128, kernel_size = (5, 5), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(strides=(2,2)))
classifier.add(Dropout(0.25))
classifier.add(Convolution2D(filters = 256, kernel_size = (5, 5), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters = 512, kernel_size = (5, 5), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(strides=(2,2)))
classifier.add(Dropout(0.25))

classifier.add(Flatten())
#classifier.add(Dense(1024, activation='relu'))
#classifier.add(Dropout(0.25))
classifier.add(Dense(512, activation='relu'))
classifier.add(Dropout(0.25))
classifier.add(Dense(256, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(47, activation='softmax'))


classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(zoom_range = 0.2,
                            height_shift_range = 0.2,
                            width_shift_range = 0.2,
                            rotation_range = 10,
                            shear_range=0.2)
#                             horizontal_flip = True # Many symbol will be confused with horizontal flip such as W and M

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

hist = classifier.fit_generator(datagen.flow(X_train, y_train, batch_size=16),
                           steps_per_epoch=1000,
                           epochs=60, #Increase this when not on Kaggle kernel
                           verbose=1,  #1 for ETA, 0 for silent
#                           validation_data=(X_test[:400,:], y_val[:400,:]))#, #For speed
                           callbacks=[annealer])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 w

Epoch 1/60
1000/1000 [==============================] - 39s 39ms/step - loss: 3.5177 - acc: 0.1043
Epoch 2/60
1000/1000 [==============================] - 36s 36ms/step - loss: 2.3693 - acc: 0.3132
Epoch 3/60
1000/1000 [==============================] - 36s 36ms/step - loss: 1.7524 - acc: 0.4731
Epoch 4/60
1000/1000 [==============================] - 35s 35ms/step - loss: 1.4519 - acc: 0.5649
Epoch 5/60
1000/1000 [==============================] - 35s 35ms/step - loss: 1.2597 - acc: 0.6133
Epoch 6/60
1000/1000 [==============================] - 35s 35ms/step - loss: 1.1647 - acc: 0.6467
Epoch 7/60
1000/1000 [==============================] - 36s 36ms/step - loss: 1.0800 - acc: 0.6695
Epoch 8/60
1000/1000 [==============================] - 35s 35ms/step - loss: 0.9705 - acc: 0.6972
Epoch 9/60
1000/1000 [==============================] - 35s 35ms/step - loss: 0.9328 - acc: 0.7077
Epoch 10/60
1000/1000 [==============================] - 36s 36ms/step - loss: 0.8870 - acc: 0.7209
Epoch 11/

In [9]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 64)        1664      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 20, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 256)         819456    
__________

In [7]:
# # save the model to disk
from sklearn.externals import joblib
filename = 'EMNIST_model.h5'
joblib.dump(classifier, filename)

['EMNIST_model.h5']

In [0]:

from google.colab import files
files.download(filename)
